In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

from pathlib import Path
import re

def main():
    # 環境変数の取得
    load_dotenv("../.env")
    # os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']
    os.environ['OPENAI_API_KEY'] = load_api_key()

    # モデル名
    MODEL_NAME = "gpt-4o-mini"

    # Indexの構築
    documents = SimpleDirectoryReader('./data/text').load_data()
    index = VectorStoreIndex.from_documents(documents)

    # Chat Engineの作成
    llm = OpenAI(model=MODEL_NAME)
    chat_engine = generate_chat_engine(llm, index)

    # チャットの例 (単発)
    # response = chat_engine.stream_chat("公共交通機関の交通費の上限は？")
    # for token in response.response_gen:
    #     print(token, end="")

    # チャットの開始
    while(True):
        message = input("メッセージを入力:")
        if message.strip()=="":
            break

        # jupyter 専用の関数なので純粋な Python では動作しないかも
        display(f"質問:{message}")

        # 質問（以下にソースコードを記述）
        response = chat_engine.stream_chat(message)

        # 回答を表示（以下にソースコードを記述）
        for token in response.response_gen:
            print(token, end="", flush=True)
        print("")

        # 引用元を表示
        for source in response.sources:
            for i, source_node in enumerate(source.raw_output.source_nodes):
                print("\033[37m")
                print(f"===== 引用情報{1+i:02d} =====================")
                print("ファイル名：", source_node.metadata["file_name"])
                print("関連度スコア:", source_node.score)
                print("テキスト：")
                print(source_node.node.text)
                print(f"===================================")
                print("\033[0m")



    print("\n---ご利用ありがとうございました！---")


# テンプレートを有効化した chatbot をつくる
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate
def generate_chat_engine(llm, index):

    sys_prompt_str = """\
事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に回答してください。
回答内でコンテキストを直接参照しないでください。
「コンテキストに基づいて」や「コンテキスト情報は」、またはそれに類するような記述は避けてください。
"""

    qa_prompt_str = """\
コンテキスト情報は以下の通りです。
---------------------
{context_str}
---------------------
事前知識ではなくコンテキスト情報を使用して、質問に回答してください。
質問: {query_str}
回答："""

    refine_prompt_str = """\
元の回答を (必要な場合のみ) 以下のコンテキストで改良する機会があります。
-----------
{context_msg}
-----------
新しいコンテキストが与えられた場合、元の回答を改良して、質問 {query_str} に適切に回答します。
コンテキストが役に立たない場合は、元の回答を再度出力します。
元の回答: {existing_answer}"""

    # テキストQAテンプレートの作成
    chat_text_qa_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=sys_prompt_str),
        ChatMessage(
            role=MessageRole.USER,
            content=qa_prompt_str),
        ]
    text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
            
    # リファインテンプレートの作成
    chat_refine_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=sys_prompt_str),
        ChatMessage(
            role=MessageRole.USER,
            content=refine_prompt_str),
        ]
    
    refine_template = ChatPromptTemplate(chat_refine_msgs)

    chat_engine = index.as_chat_engine(
        chat_mode="openai",
        llm=llm,
        similarity_top_k=3,
        text_qa_template=text_qa_template,
        refine_template=refine_template,
    )

    return chat_engine


# 上手く鍵を読み取れない時があるので代わりに以下で無理やり取得しています ...
def load_api_key():
    api_key = None

    # 1. ../.env を読み取る
    env_file_path = Path().resolve().parent.resolve() / ".env"
    if env_file_path.is_file():
        try:
            load_dotenv(env_file_path)
        except:
            raise Exception("Found .env file but failed to load dotenv file! Please install python-dotenv module.")

    # 2. API_KEY を読み取る
    api_key = os.environ.get("API_KEY", None)

    # 3. API_KEY の中身チェック
    api_file_path = Path(api_key).expanduser()
    if (api_file_path.is_absolute() and api_file_path.is_file()):
        with open(api_file_path, "r") as f:
            api_key = f.read().strip()

    # 4. キーの簡易チェック
    if re.match(r"^sk\-.*$", api_key) is None:
        raise Exception("Failed to load api key!")
        
    return api_key

    
if __name__ == "__main__":
    main()